In [1]:
import sys

sys.path.append("../modules")

top_dir = "/Users/pictomo/Repositories/SequentialCTA_experiment"

In [2]:
# prepare data

from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

labels = [
    "Airplane",
    "Car",
    "Bird",
    "Cat",
    "Deer",
    "Dog",
    "Frog",
    "Horse",
    "Ship",
    "Truck",
]
label_to_index = {label: i for i, label in enumerate(labels)}

# normalize
# x_train, x_test = x_train / 255.0, x_test / 255.0

In [3]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam

def provide_model(optimizer="adam"):
    input_layer = Input(shape=(32, 32, 3))

    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)

    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    output = Dense(10, activation="softmax")(x)

    model = Model(input_layer, output)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
# prepare haio
from haio import HAIOClient, OpenAI_IO, Gemini_IO, Bedrock_IO, QuestionTemplate
from virtual_human_io import VirtualHuman_IO

virtual_human_io = VirtualHuman_IO()
gemini_io = Gemini_IO()
openai_io = OpenAI_IO()
llama_io = Bedrock_IO("us.meta.llama3-2-90b-instruct-v1:0")
nova_io = Bedrock_IO("us.amazon.nova-lite-v1:0")

haio_client = HAIOClient(
    filepath=f"{top_dir}/notebooks",
    human_io=virtual_human_io,
    openai_io=openai_io,
    gemini_io=gemini_io,
    llama_io=llama_io,
    nova_io=nova_io,
    # # claude_io=claude_io,
)

question_template: QuestionTemplate = QuestionTemplate(
    title="CIFAR-10 Image Classification",
    description="Classify the image using the CIFAR-10 dataset",
    question=[
        {"tag": "img", "src": 0},
        {
            "tag": "h1",
            "value": "Choose the label that best describes the image from the options.",
        },
        {"tag": "p", "value": " ".join(labels)},
    ],
    answer={
        "type": "select",
        "options": labels,
    },
)

In [12]:
from haio import img_to_url
from vec_to_img import vec_to_img
import numpy as np
import hashlib
import json


def hash_data(src: any) -> str:  # same as haio_hash
    return hashlib.md5(json.dumps(src, sort_keys=True).encode()).hexdigest()


data_to_label_index = {hash_data(img.tolist()): y_train[i] for i, img in enumerate(x_train)}


def provide_label_collect(img_list) -> np.ndarray:
    return np.array([data_to_label_index[hash_data(img.tolist())] for img in img_list])


async def provide_label(img_list: list, execution_config: dict) -> HAIOClient.MethodReturn:
    asked_questions = []
    for img in img_list:
        img_url = img_to_url(img_data=vec_to_img(img), mime_type="image/png")
        data_list = [img_url]
        asked_questions.append(
            haio_client.ask(
                question_template=question_template,
                data_list=data_list,
            )
        )
    answer_info = await haio_client.wait(
        asked_questions=asked_questions, execution_config=execution_config
    )
    return answer_info

In [13]:
import copy

async def active_learn(
    execution_config,
    data,
    data_size=10000,
    retrain=True,
    initial_size=1000,
    query_size=100,
    n_steps=40,
    epochs=10,
):
    result = {
        "question_count": [0],
        "human_count": [0],
        "collect_count": [0],
        "accuracy": [0.1],
        "loss": [2.302585092994],
    }
    print("Active Learning Start")

    x_train = copy.deepcopy(data[:data_size])

    # 最初の1,000枚を使用する
    model = provide_model()

    initial_indices = np.arange(initial_size)
    x_labeled = x_train[initial_indices]

    # ラベリングと学習
    answer_info = await provide_label(x_labeled, execution_config)
    y_labeled = np.array(
        [np.array([label_to_index[ans]]) for ans in answer_info["answer_list"]]
    )
    collect_labels = provide_label_collect(x_labeled)

    result["question_count"].append(result["question_count"][-1] + initial_size)
    result["human_count"].append(
        result["human_count"][-1] + answer_info["add_human_assign"]
    )
    collect_count = result["collect_count"][-1]
    for i in range(initial_size):
        if y_labeled[i][0] == collect_labels[i][0]:
            collect_count += 1
    result["collect_count"].append(collect_count)

    model.fit(x_labeled / 255.0, y_labeled, epochs=epochs, verbose=1)
    loss, acc = model.evaluate(x_test / 255.0, y_test)

    result["accuracy"].append(acc)
    result["loss"].append(loss)

    # 残りの9,000枚を未使用として保持
    remaining_indices = np.arange(initial_size, data_size)
    x_unlabeled = x_train[remaining_indices]

    # Active Learningのループ
    for step in range(n_steps):
        print(f"Step {step + 1}/{n_steps}")
        # 未使用データに対して予測を行う
        predictions = model.predict(x_unlabeled / 255.0)
        # 予測の不確実性を計算（確信度の低い順に選ぶ）
        uncertainties = np.max(predictions, axis=1)
        query_indices = np.argsort(uncertainties)[:query_size]

        # 新しく選ばれたデータをラベル付きデータに追加
        new_samples = x_unlabeled[query_indices]
        answer_info = await provide_label(
            new_samples, execution_config=execution_config
        )
        new_labels = np.array(
            [
                np.array([label_to_index[answer]])
                for answer in answer_info["answer_list"]
            ]
        )
        collect_labels = provide_label_collect(new_samples)

        result["question_count"].append(result["question_count"][-1] + query_size)
        result["human_count"].append(
            result["human_count"][-1] + answer_info["add_human_assign"]
        )
        collect_count = result["collect_count"][-1]
        for i in range(query_size):
            if new_labels[i][0] == collect_labels[i][0]:
                collect_count += 1
        result["collect_count"].append(collect_count)

        x_labeled = np.concatenate([x_labeled, new_samples])
        y_labeled = np.concatenate([y_labeled, new_labels])

        # 未使用データから選ばれたデータを除去
        x_unlabeled = np.delete(x_unlabeled, query_indices, axis=0)

        # モデルをFine-Tuning
        if retrain:
            model = provide_model()
        model.fit(x_labeled / 255.0, y_labeled, epochs=epochs, verbose=1)

        # テストセットでの評価
        loss, acc = model.evaluate(x_test / 255.0, y_test)

        result["accuracy"].append(acc)
        result["loss"].append(loss)

    return result

In [ ]:
from ensure_dir import ensure_dir
from datetime import datetime
import json

execution_config_list = [
    ({"method": "sequential_cta_1", "quality_requirement": 0.975}, "cta1"),
    ({"method": "sequential_gta_1", "quality_requirement": 0.975}, "gta1"),
    # (
    #     {"method": "sequential_cta_2", "quality_requirement": 0.975, "sample_size": 250},
    #     "cta2",
    # ),
    # (
    #     {"method": "sequential_gta_2", "quality_requirement": 0.975, "sample_size": 250},
    #     "gta2",
    # ),
    # ({"method": "sequential_cta_3", "quality_requirement": 0.975}, "cta3"),
    # ({"method": "sequential_gta_3", "quality_requirement": 0.975}, "gta3"),
]

for execution_config, name in execution_config_list:
    ensure_dir(f"{top_dir}/results/expr2")
    haio_client.del_cache_use_hist()
    haio_client.reset_state()
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")
    result = await active_learn(
        execution_config=execution_config, data=x_train, retrain=True, initial_size=1000, query_size=100, n_steps=40, epochs=20
    )
    print(result)
    with open(f"{top_dir}/results/expr2/{name}_{execution_config["quality_requirement"]}_{timestamp}.json", "w") as f:
        json.dump(result, f)